# Google Search Tool with Strands Agents SDK by Markus Bestehorn (bestem@amazon.com)

This notebook demonstrates how to create a Google search tool for the Strands Agents SDK and implement an agent that uses it to answer questions requiring web search.

## Prerequisites

- Google Custom Search API key stored in `google-api-key.txt`
- Google Custom Search Engine ID (we'll use a programmable search engine)
- Strands Agents SDK installed
- AWS credentials configured for Bedrock

For the Custom Search API Key and the Engine ID, there are dedicated sections below on how to obtain them. Using AWS credentials for Bedrock with Strands Agents and which permissions are required for accessing Bedrock is outlined [here](https://strandsagents.com/0.1.x/user-guide/concepts/model-providers/amazon-bedrock/).

### Getting a Custom Search Engine API Key
For the Google Customer Search Engine (CSE) API Key, you need a Google account, e.g., an account with Gmail. You can create such an account by clicking [here](https://accounts.google.com/signin) and selecting "Create Account". If you already have an account or after creating one, log-in to this account and then:

 - browse to the URL "[https://developers.google.com/custom-search/v1/introduction](https://developers.google.com/custom-search/v1/introduction)"
 - There is a button in the middle of the screen labeled "Get a Key". Click the button.
 - Enter a Project name, e.g., "StarndsGoogleSearchTool".
 - The site will take a few seconds to generate a key. Click the "Show Key" button and copy the API key to the clipboard.
 - Open a text editor and create new file. Paste the API key from the clipboard to the file. Save the file on your harddrive.
 - Note down the location/path and name of the file and enter it in the cell below.

In [ ]:
## Paste the full path of the file that contains your Google API Key into the below line
GOOGLE_API_KEY_FILE = "./google-api-key.txt"

**Important**: I have deliberately moved this key to a separate file instead of having it directly in the notebook. Credentials such as this API should not be stored directly in notebooks to avoid the chance of such credentials being accidentally stored in code repositories. If you choose to store the API Key in a file that is in the repository for this notebook, make sure the file is in the .gitignore file.

### Creating a Custom Search Engine ID
Next you need a custom search engine (CSE) ID. The CSE ID is used to customize your searches. In our case, we will stick to text-only results, but this would be a way to further customize the search that the agent can use or modify the results. For instance, you could expand the searches to images as well by including images or restrict the search to specific websites (e.g., wikipedia only), but for this simple example, we will stick to a very standard setup. To complete this part of the setup, execute the following steps:

 - if you are not logged into your Google account, please do so as described above.
 - Open the URL "[https://programmablesearchengine.google.com/controlpanel/create](https://programmablesearchengine.google.com/controlpanel/create)" in your browser
 - Select the option to search the whole web and keep everything else as it is.
 - Confirm that you are not a robot
 - Click "Create" at the bottom
 - After a few seconds, there will be a confirmation page that your CSE has been created. The page also displays a code snippet that outlines how to integrate your CSE into your web page, but this is not what we want.
 - On the left-hand menu, there is an option to go back to all of your CSEs, alternatively, you can also just click on this link: [https://programmablesearchengine.google.com/controlpanel/all](https://programmablesearchengine.google.com/controlpanel/all)
 - Your new CSE should be shown here. Click on it and there is an entry "Search Engine ID". Copy this ID to your clipboard.
 - Open a text editor and create a new file. Paste the Search Engine ID into this file. Save the file to your harddrive.
 - Note down the location/path and name of the file and enter it in the cell below.

The CSE ID will be loaded from this file below.

In [ ]:
SEARCH_ENGINE_ID_FILE = "./google-search_engine-id.txt"

## 1. Install Required Dependencies
This demo has a few dependencies. The cell below only needs to be executed once for every environment where this notebook is executed. It install the dependencies as needed.

In [ ]:
# Install required packages
!pip install strands-agents strands-agents-tools strands-agents-builder google-api-python-client -q

## 2. Import Required Libraries and Setup

In [ ]:
import boto3
import os
from googleapiclient.discovery import build
from strands import Agent, tool
from strands.models import BedrockModel
import json
from typing import List, Dict

## 3. Load Google API Key and Setup

In [ ]:
# Load Google API key from file
try:
    with open(GOOGLE_API_KEY_FILE, 'r') as f:
        google_api_key = f.read().strip()
    print("Google API key loaded successfully!")
except FileNotFoundError:
    print(f"Error: {GOOGLE_API_KEY_FILE} file not found. Please create this file as described above.")
    google_api_key = None

# Load Google API key from file
try:
    with open(SEARCH_ENGINE_ID_FILE, 'r') as f:
        search_engine_id = f.read().strip()
    print("Search Engine ID loaded successfully!")
except FileNotFoundError:
    print(f"Error: {SEARCH_ENGINE_ID_FILE} file not found. Please create this file as described above.")
    search_engine_id = None

## 4. Setup AWS Bedrock Model
Next we define how we will use Bedrock. If you want to use a different AWS CLI Profile name than the one shown here, change the corde accoridingly.

In [ ]:
# Setup AWS session and Bedrock model
session = boto3.Session(
    profile_name='default'  # Optional: Use a specific profile
)

# Create a Bedrock model with the custom session
bedrock_model = BedrockModel(
    model_id="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    boto_session=session
)

print("Bedrock model configured successfully!")

## 5. Create Google Search Tool

In [ ]:
@tool
def google_search(query: str, num_results: int = 3) -> str:
    """
    Search Google and return top results with titles, URLs, and snippets.
    
    Args:
        query: The search query string
        num_results: Number of results to return (default: 3, max: 10)
    
    Returns:
        Formatted string with search results including titles, URLs, and descriptions
    """
    if not google_api_key:
        return f"Error: Google API key not available. Please check {GOOGLE_API_KEY_FILE} file."

    if not search_engine_id:
        return f"Error: Search Engine ID not available. Please check {SEARCH_ENGINE_ID_FILE} file."
    
    try:
        # Build the Custom Search service
        service = build("customsearch", "v1", developerKey=google_api_key)
        
        # Perform the search
        result = service.cse().list(
            q=query,
            cx=search_engine_id,
            num=min(num_results, 10)  # Google API allows max 10 results per request
        ).execute()
        
        # Format the results
        if 'items' not in result:
            return f"No search results found for query: '{query}'"
        
        formatted_results = []
        for i, item in enumerate(result['items'][:num_results], 1):
            title = item.get('title', 'No title')
            link = item.get('link', 'No URL')
            snippet = item.get('snippet', 'No description available')
            
            formatted_result = f"**Result {i}:**\n"
            formatted_result += f"Title: {title}\n"
            formatted_result += f"URL: {link}\n"
            formatted_result += f"Description: {snippet}\n"
            
            formatted_results.append(formatted_result)
        
        return "\n".join(formatted_results)
        
    except Exception as e:
        return f"Error performing Google search: {str(e)}"

print("Google search tool created successfully!")

## 6. Test the Google Search Tool Directly

In [ ]:
# Test the search tool directly
test_query = "artificial intelligence latest news 2024"
search_results = google_search(test_query)
print(f"Direct search test for: '{test_query}'")
print("=" * 50)
print(search_results)

## 7. Create Agent with Google Search Tool

In [ ]:
# Create an agent equipped with the Google search tool
search_agent = Agent(
    system_prompt="""You are a helpful research assistant equipped with Google search capabilities. 
    When users ask questions that require current information, recent news, or facts that you might not know, 
    use the google_search tool to find relevant information. 
    Always cite your sources and provide URLs when presenting information from search results.
    Be concise but informative in your responses.""",
    tools=[google_search],
    model=bedrock_model,
)

print("Search agent created successfully!")

## 8. Demo: Agent Using Google Search Tool

In [ ]:
# Test questions that require web search
test_questions = [
    "What are the latest developments in AI in 2024?",
    "What is the current weather in London?",
    "Who won the latest Nobel Prize in Physics?"
]

for question in test_questions:
    print(f"\n{'='*60}")
    print(f"Question: {question}")
    print(f"{'='*60}")
    
    response = search_agent(question)
    print(f"Agent's response:")
    print(response)
    print("\n")

## 9. Advanced Example: Comparative Research

In [ ]:
# Example of using the agent for comparative research
research_question = "Compare the latest iPhone vs Samsung Galaxy phones released in 2024"

print(f"Research Question: {research_question}")
print("=" * 70)

response = search_agent(research_question)
print(f"Research Assistant's Response:")
print(response)

## 10. Tool Performance Analysis

In [ ]:
import time

# Analyze the performance of our search tool
def analyze_search_performance(query: str):
    """Analyze the performance of our Google search tool"""
    print(f"Analyzing performance for query: '{query}'")
    
    # Test direct tool performance
    start_time = time.time()
    direct_result = google_search(query)
    direct_time = time.time() - start_time
    
    # Test agent performance (includes AI processing)
    start_time = time.time()
    agent_result = search_agent(f"Search for: {query}")
    agent_time = time.time() - start_time
    
    print(f"\nPerformance Results:")
    print(f"Direct tool call: {direct_time:.2f} seconds")
    print(f"Agent with tool: {agent_time:.2f} seconds")
    print(f"AI processing overhead: {agent_time - direct_time:.2f} seconds")
    
    return direct_time, agent_time

# Run performance analysis
analyze_search_performance("Python programming best practices")

## 12. Error Handling and Edge Cases

In [ ]:
# Test error handling and edge cases
edge_cases = [
    "",  # Empty query
    "asdfghjklqwertyuiopzxcvbnm123456789",  # Nonsensical query
    "site:nonexistentwebsite12345.com information",  # No results expected
]

for case in edge_cases:
    print(f"\nTesting edge case: '{case}'")
    print("-" * 40)
    
    try:
        if case:  # Only test non-empty strings with the agent
            response = search_agent(f"Please search for: {case}")
            print(f"Agent handled gracefully: {response[:100]}...")
        else:
            direct_response = google_search(case)
            print(f"Direct tool response: {direct_response}")
    except Exception as e:
        print(f"Error caught: {e}")

## Summary and Next Steps

This notebook has demonstrated:

1. **Google Search Tool Creation**: Built a custom `@tool` function that integrates with Google Custom Search API
2. **Agent Integration**: Created an agent that can intelligently use the search tool
3. **Real-world Testing**: Tested with various queries including current events and comparative research
4. **Performance Analysis**: Measured tool and agent response times
5. **Error Handling**: Demonstrated graceful handling of edge cases and errors

### Key Features of Our Google Search Tool:
- Returns top 3 search results by default (configurable)
- Includes title, URL, and description for each result
- Proper error handling for API failures
- Type hints for proper parameter passing
- Integration with Strands Agents SDK

### Potential Enhancements:
- Add support for different search types (images, news, etc.)
- Implement caching to avoid duplicate API calls
- Add filtering options (date range, site-specific searches)
- Implement search result summarization
- Add support for multiple search engines

### Production Considerations:
- API key security and rotation
- Rate limiting and quota management
- Result caching for performance
- Monitoring and logging
- Content filtering and safety checks

Happy searching with your Strands Agent! 🔍🤖